# DISPONIBILIDAD DE SUELO URBANIZABLE

## 1. DEFINICIÓN 

Tal como hemos mencionado anteriormente en relación con la climatología, en esta fase del estudio nos enfocaremos en analizar la media de la disponibilidad de suelo urbanizable en las regiones de Cantabria y Andalucía, para ello, se han tenido en cuenta dos variables, precio y metros cuadrados.

Con esta última parte del proyecto, llegaremos a la conclusión sobre la ubicación más adecuada para establecer la empresa de nuestro cliente, teniendo en cuenta el precio medio y la disponibilidad de suelo urbanizable en cada zona.

Para la recopilación de datos, hemos utilizado el portal inmobiliario Fotocasa, donde hemos filtrado los terrenos disponibles por ubicación, así como sus precios. La extracción de la información se ha realizado mediante un proceso de web scraping, capturando el código HTML y guardándolo en un archivo TXT (todos los archviso obtenidos están en la carpeta html_fotocasa). En este análisis, se han considerado las 10 primeras páginas de resultados de cada búsqueda, tanto para Cantabria como para Andalucía.

In [1]:
#importamos las librerías necesarias
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import requests


## 2. SUELO URBANIZABLE ANDALUCÍA

Para éste apartado, se ha decidio crear funciones, con la finalidad de no tener que repertir el código 10 veces por cada Comunidad Autónoma, de ésta forma tenemos un código más eficiente y legible, además de escalable. 

In [2]:
#Definición de listas globales: 
andalucia_precios = []
andalucia_metros_cuadrados = []


#Esta función se encarga de leer el archivo .txt que contiene el código HTML de la página de resultados Fotocasa. Utiliza BeautifulSoup (from bs4 import BeautifulSoup) para analizar el HTML y devolver un objeto soup, que permite navegar fácilmente por el árbol de etiquetas HTML.
def obtener_soup(nombre_txt):
    with open(nombre_txt) as fp:
        return bs4(fp)
    
#Esta función recorre todos los elementos del HTML que contienen el precio de un terreno (etiquetas span con la clase re-CardPrice). Si el texto del precio no es "A consultar", lo agrega a la lista andalucia_precios.

def obtener_precios_andalucia(soup):
    for precio in soup.select("span.re-CardPrice"):
        if not precio.text == "A consultar":
            andalucia_precios.append(precio.text)

#Esta función extrae los metros cuadrados de la propiedad, que están en elementos span con la clase re-CardFeaturesWithIcons-feature-icon--surface. Para este apartado hacemos dos funciones, ya que la clase en el cógido html cambia a partir de la 8.

def obtener_metros_cuadrados_andalucia_del_1_al_7(soup):
    for metro_cuadrado in soup.select("span.re-CardFeaturesWithIcons-feature-icon.re-CardFeaturesWithIcons-feature-icon--surface"):
        andalucia_metros_cuadrados.append(metro_cuadrado.text)

#Similar a la anterior, pero extrae los metros cuadrados de una clase diferente: re-CardFeatures-item. Del 8 al 10

def obtener_metros_cuadrados_andalucia_del_8_al_10(soup):
    for metro_cuadrado in soup.select("li.re-CardFeatures-item.re-CardFeatures-feature"):
        andalucia_metros_cuadrados.append(metro_cuadrado.text)

#A continuación, se recorren los archivos de resultados del portal Fotocasa ubicados en la caroeta html_fotocasa utilizando dos bucles for.

for i in range(1, 8):
    soup = obtener_soup(f"html_fotocasa/andalucia_{i}.txt")

    obtener_precios_andalucia(soup)

    obtener_metros_cuadrados_andalucia_del_1_al_7(soup)

for i in range(8, 11):
    soup = obtener_soup(f"html_fotocasa/andalucia_{i}.txt")

    obtener_precios_andalucia(soup)

    obtener_metros_cuadrados_andalucia_del_8_al_10(soup)

#En éste apratado, se procesan los precios: se eliminan los caracteres no numéricos (' €' y '.'), convirtiendo los valores en enteros. Luego, se crea un DataFrame para almacenar los precios, y se calcula la media de los precios con el método mean().
andalucia_precios = [int(valor.replace(' €', '').replace('.', '')) for valor in andalucia_precios]
df_precios_andalucia = pd.DataFrame({ "precios_andalucia": andalucia_precios })
media_precios_andalucia = df_precios_andalucia.loc[:, "precios_andalucia"].mean()
print(media_precios_andalucia)

andalucia_metros_cuadrados = [int(valor.replace('m²', '')) for valor in andalucia_metros_cuadrados]
df_metros_cuadrados_andalucia = pd.DataFrame({ "metros_cuadrados_andalucia": andalucia_metros_cuadrados })
media_metros_cuadrados_andalucia = df_metros_cuadrados_andalucia.loc[:, "metros_cuadrados_andalucia"].mean()
print(media_metros_cuadrados_andalucia)

217746.0172413793
16782.324444444443


## 3. SUELO URBANIZABLE CANTABRIA

Repetimos los mismos pasos para Cantabria, con la diferencia que en éste apartado se mantienen las clases tanto para precios como para metros cuadrados. 

In [3]:
cantabria_precios = []
cantabria_metros_cuadrados = []

def obtener_soup(nombre_txt):
    with open(nombre_txt) as fp:
        return bs4(fp)

def obtener_precios_cantabria(soup):
    for precio in soup.select("span.re-CardPrice"):
        if not precio.text == "A consultar":
            cantabria_precios.append(precio.text)

def obtener_metros_cuadrados_cantabria(soup):
    for metro_cuadrado in soup.select("li.re-CardFeatures-item.re-CardFeatures-feature"):
        cantabria_metros_cuadrados.append(metro_cuadrado.text)


for i in range(1, 11):
    soup = obtener_soup(f"html_fotocasa/cantabria_{i}.txt")

    obtener_precios_cantabria(soup)

    obtener_metros_cuadrados_cantabria(soup)


cantabria_precios = [int(valor.replace(' €', '').replace('.', '')) for valor in cantabria_precios]
df_precios_cantabria = pd.DataFrame({ "precios_cantabria": cantabria_precios })
media_precios_cantabria = df_precios_cantabria.loc[:, "precios_cantabria"].mean()
print(media_precios_cantabria)

cantabria_metros_cuadrados = [int(valor.replace('m²', '')) for valor in cantabria_metros_cuadrados]
df_metros_cuadrados_cantabria = pd.DataFrame({ "metros_cuadrados_cantabria": cantabria_metros_cuadrados })
media_metros_cuadrados_cantabria = df_metros_cuadrados_cantabria.loc[:, "metros_cuadrados_cantabria"].mean()
print(media_metros_cuadrados_cantabria)


3407443.8745980705
7902.942028985507


## 4. CONCLUSIÓN

Una vez obtenidas las medias tanto de precios como de metros cuadrados podemos observar que los resultados son los siguientes:
###  **ANDALUCÍA**
La media de los precios obtenido es de: 217.746 euros

La media de los metros cuadrados es de: 16.782 m²

###  **CANTABRIA**
La media de los precios obtenidos es de: 3.407.443 euros

La media de los metros cuadrados es de:  7.902m²

Según los datos obtenidos, Andalucía es la zona que mejores oportunidades ofrece a nuestro cliente, ya que la media es mucho más baja respecto a Cantabria y hay una mayor disponibilidad de metros cuadrados. 